# Необходимые библиотеки и настройки

In [3]:
# ! pip install scipy
# ! pip install numpy
# ! pip install matplotlib

In [4]:
from scipy import stats
import scipy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Функции для расчета стоимости опционов по разным моделям

In [5]:
def black_scholes(S, X, t, r, sigma, option_type='call'):
    """ Рассчитывает стоимость опциона по модели Блэка-Шоулза на действительных числах """
    d1 = (np.log(S / X) + (r + 0.5 * sigma ** 2) * t) / \
        (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    if option_type == 'call':
        price = S * stats.norm.cdf(d1) - X * \
            np.exp(-r * t) * stats.norm.cdf(d2)
    elif option_type == 'put':
        pass
    return price

In [6]:
def black_scholes_fuzzy(S, X, t, r, sigma, option_type='call'):
    """ Рассчитывает стоимость опциона по модели Блэка-Шоулза на нечетких числах """
    S = np.array(S)
    r = np.array(r)
    sigma = np.array(sigma)
    d1 = (np.log(S/X) + (r+0.5*sigma ** 2)*t) / (sigma * np.sqrt(t))
    d2 = d1-sigma*np.sqrt(t)
    if option_type == 'call':
        price = S * stats.norm.cdf(d1) - X * \
            np.exp(-r * t) * stats.norm.cdf(d2)
    elif option_type == 'put':
      pass
    return price

In [7]:
def binomial_model(S, X, t, r, sigma, N, option_type='call'):
    """ Рассчитывает стоимость опциона по биномиальной модели на действительных числах """
    dt = t / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    stock_price = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            stock_price[j, i] = S * (u ** (i - j)) * (d ** j)
    option_price = np.zeros((N + 1, N + 1))
    if option_type == 'call':
        option_price[:, N] = np.maximum(stock_price[:, N] - X, 0)
        for i in range(N - 1, -1, -1):
            for j in range(i + 1):
                option_price[j, i] = (p * option_price[j, i + 1] + (1 - p)
                                      * option_price[j + 1, i + 1]) * np.exp(-r * dt)
    elif option_type == 'put':
      pass
    return option_price[0, 0]

In [8]:
def binomial_model_fuzzy(S, X, t, r, sigma, N, option_type='call'):
    """ Рассчитывает стоимость опциона по биномиальной модели на нечетких числах """
    S = np.array(S)
    r = np.array(r)
    sigma = np.array(sigma)
    dt = t/N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1/(u[::-1])
    p = ((np.exp(r * dt) - d) / (u - d))[::-1]
    stock_price = np.zeros((N + 1, N + 1, 3))
    for i in range(N + 1):
        for j in range(i + 1):
            stock_price[j, i, :] = S * (u ** (i - j)) * (d ** j)
    option_price = np.zeros((N + 1, N + 1, 3))
    option_price[:, N, :] = np.maximum(stock_price[:, N, :] - X, 0)
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_price[j, i, :] = (p * option_price[j, i + 1] + (
                1 - p) * option_price[j + 1, i + 1]) * np.exp(-r * dt)
    return option_price[0, 0]

# Тестовые параметры для моделей

In [11]:
# цена акции, страйк, время до истечения срока действия опциона, безрисковая ставка процента, волатильность базисного актива, число шагов (только для биномиальной модели)
black_scholes_args = [10, 10, 3, 0.1, 0.2]
black_scholes_fuzzy_args = [(9, 10, 11), 10, 3, (0.05, 0.1, 0.15), (0.1, 0.2, 0.3)]
binomial_args = [10, 10, 3, 0.1, 0.2, 10]
binomial_fuzzy_args = [(9, 10, 11), 10, 3, (0.05, 0.1, 0.15), (0.1, 0.2, 0.3), 5]

# Пример использования функций для расчетов стоимости опционов

In [12]:
black_scholes(*black_scholes_args)

2.907490770915932

In [13]:
black_scholes_fuzzy(*black_scholes_fuzzy_args)

array([0.82402741, 2.90749077, 4.94632037])

In [14]:
binomial_model(*binomial_args)

2.872615808787606

In [15]:
binomial_model_fuzzy(*binomial_fuzzy_args)

array([ 0.08929216,  2.91309905, 11.3049992 ])